<a href="https://colab.research.google.com/github/kumarmohit0911/AAA/blob/main/tamato_disease_over_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json, os

kaggle_creds = {
    "username": "kumarmohit0911",
    "key": "KGAT_70d781b0a7eed54b863a6bcb5ab50e9c"
}

os.makedirs("/root/.kaggle", exist_ok=True)

with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_creds, f)

os.chmod("/root/.kaggle/kaggle.json", 0o600)

print("kaggle.json created successfully")


In [2]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kaustubhb999/tomatoleaf")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'tomatoleaf' dataset.
Path to dataset files: /kaggle/input/tomatoleaf


In [4]:
import os
DATA_DIR = '/kaggle/input/tomatoleaf/tomato'

image_paths = []
labels = []

train_data = os.path.join(DATA_DIR,"train")
val_data = os.path.join(DATA_DIR,'val')
for disease_type in os.listdir(train_data):
  disease_path = os.path.join(train_data,disease_type)
  if disease_type == 'Tomato___Bacterial_spot':
    for img in os.listdir(disease_path):
      image_paths.append(os.path.join(disease_path,img))
      labels.append(0)
  elif disease_type == 'Tomato___Early_blight':
    for img in os.listdir(disease_path):
      image_paths.append(os.path.join(disease_path,img))
      labels.append(1)
  elif disease_type == 'Tomato___healthy':
    for img in os.listdir(disease_path):
      image_paths.append(os.path.join(disease_path,img))
      labels.append(2)
  elif disease_type == 'Tomato___Late_blight':
    for img in os.listdir(disease_path):
      image_paths.append(os.path.join(disease_path,img))
      labels.append(3)
  elif disease_type == 'Tomato___Leaf_Mold':
    for img in os.listdir(disease_path):
      image_paths.append(os.path.join(disease_path,img))
      labels.append(4)
  elif disease_type == 'Tomato___Septoria_leaf_spot':
    for img in os.listdir(disease_path):
      image_paths.append(os.path.join(disease_path,img))
      labels.append(5)
  elif disease_type == 'Tomato___Spider_mites Two-spotted_spider_mite':
    for img in os.listdir(disease_path):
      image_paths.append(os.path.join(disease_path,img))
      labels.append(6)
  elif disease_type == 'Tomato___Target_Spot':
    for img in os.listdir(disease_path):
      image_paths.append(os.path.join(disease_path,img))
      labels.append(7)
  elif disease_type == 'Tomato___Tomato_mosaic_virus':
    for img in os.listdir(disease_path):
      image_paths.append(os.path.join(disease_path,img))
      labels.append(8)
  elif disease_type == 'Tomato___Tomato_Yellow_Leaf_Curl_Virus':
    for img in os.listdir(disease_path):
      image_paths.append(os.path.join(disease_path,img))
      labels.append(9)

In [5]:
# for train data
import random
#zip everything
combined = list(zip(image_paths,labels))
#shuffle
random.shuffle(combined)
#unzip
image_paths,labels = zip(*combined)
image_paths,labels = list(image_paths),list(labels)

In [6]:
import torch
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.optim as optim

In [14]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image

my_transform = transforms.Compose([
    transforms.Resize((62,62)),
    # transforms.RandomRotation(degrees=45),
    # transforms.ColorJitter(brightness=0.5,contrast=0.5),
    # transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor()

])

In [9]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

class CustomDataset(Dataset):
  def __init__(self,image_paths,labels,transforms=my_transform):
    self.image_paths = image_paths
    self.labels = labels
    self.transforms= transforms
  def __len__(self):
    return len(self.image_paths)
  def __getitem__(self, index):
    image = self.image_paths[index]
    image = Image.open(image).convert("RGB")
    if self.transforms:
      image = self.transforms(image)
    # get label
    label = self.labels[index]
    return image , label



In [10]:
dataset = CustomDataset(image_paths, labels)
# dataset_val = CustomDataset(image_paths_val,labels_val)
train_loader = DataLoader(dataset,batch_size=128,num_workers=2,shuffle=True,pin_memory=True)
# test_loader = DataLoader(dataset_val,batch_size=128,num_workers=2,shuffle=False)
# test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [11]:
class MyNN(nn.Module):
  def __init__(self,num_feature):
    super().__init__()
    self.features = nn.Sequential(
        nn.Conv2d(num_feature,32,kernel_size=3,padding = "same"),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(32,64,kernel_size=3,padding="same"),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(64,128,kernel_size=3,padding="same"),
        nn.ReLU(),
        nn.BatchNorm2d(128),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.classifier = nn.Sequential(
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),

        nn.Linear(128,32),
        nn.ReLU(),
        nn.Dropout(p=0.4),
        nn.Linear(32,10)

    )

  def forward(self,x):
    x= self.features(x)
    x= self.classifier(x)
    return(x)


In [18]:
from backcall.backcall import Parameter
#setting epochs
epochs = 50
learning_rate = 1e-4

#instatiate model
model=MyNN(3)
model = model.to(device)
#loss function
criterion = nn.CrossEntropyLoss()
#optimizer
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [19]:
for epoch in range(epochs):
  total_loss_per_epoch = 0
  for images,label in train_loader:
    images,label = images.to(device),label.to(device)

    #forward pass
    output = model(images)

    #loss calculation
    loss = criterion(output,label)


    #backward pass
    optimizer.zero_grad()
    loss.backward()

    #update grads
    optimizer.step()
    total_loss_per_epoch+=loss.item()
    avg_loss = total_loss_per_epoch/len(train_loader)
  print(f'EPOCH No : {epoch+1} --> Loss : {avg_loss}')
  print('_'*50)

EPOCH No : 1 --> Loss : 2.061888869804672
__________________________________________________
EPOCH No : 2 --> Loss : 1.7229450669469712
__________________________________________________
EPOCH No : 3 --> Loss : 1.5093482536605642
__________________________________________________
EPOCH No : 4 --> Loss : 1.3360745786111565
__________________________________________________
EPOCH No : 5 --> Loss : 1.2027645473238788
__________________________________________________
EPOCH No : 6 --> Loss : 1.0782333431364615
__________________________________________________
EPOCH No : 7 --> Loss : 0.9742671251296997
__________________________________________________
EPOCH No : 8 --> Loss : 0.8995587146734889
__________________________________________________
EPOCH No : 9 --> Loss : 0.8268085416359238
__________________________________________________
EPOCH No : 10 --> Loss : 0.7703898473630978
__________________________________________________
EPOCH No : 11 --> Loss : 0.6958047996593427
________________

In [20]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
Tesla T4


In [21]:
!nvidia-smi


Mon Jan 12 06:59:05 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             34W /   70W |     702MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----